<a href="https://colab.research.google.com/github/Muhtasham/tajik-nlp/blob/main/Tajik_ASR_with_Whisper_FLEURS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-d8_woi5n
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-d8_woi5n
     |████████████████████████████████| 4.9 MB 14.6 MB/s 
     |████████████████████████████████| 120 kB 72.6 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1174851 sha256=9a2543a7a0e01f9870b1690144c278e408d151b5e716a33cd8745fce6ea451f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-avslkfjm/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

# Loading the Fleurs dataset

Select the language of the Fleur dataset to download. Please note that the transcription and translation performance varies widely depending on the language. Appendix D.2 in the paper contains the performance breakdown by language.

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="ko_kr",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=39, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Tajik (tg_tj)


In [ ]:
class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            with urllib.request.urlopen(url) as source, open(tar_path, "wb") as output:
                with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
                    while True:
                        buffer = source.read(8192)
                        if not buffer:
                            break

                        output.write(buffer)
                        loop.update(len(buffer))

        labels = {}
        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]
                    

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        
        return (audio, text)

In [ ]:
dataset = Fleurs(lang, subsample_rate=10)  # subsample 10% of the dataset for a quick demo

  0%|                                              | 0.00/1.79G [00:00<?, ?iB/s]

# Running inference on the dataset using a medium Whisper model

The following will take a few minutes to transcribe and translate utterances in the dataset.

In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 148MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

In [ ]:
references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]
    
    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

  0%|          | 0/60 [00:00<?, ?it/s]


100%|██████████| 1662/1662 [00:24<00:00, 67.34frames/s]

100%|██████████| 1662/1662 [00:06<00:00, 269.10frames/s]

100%|██████████| 1020/1020 [00:16<00:00, 62.92frames/s]

100%|██████████| 1020/1020 [00:03<00:00, 279.35frames/s]

100%|██████████| 1374/1374 [00:07<00:00, 187.22frames/s]

100%|██████████| 1374/1374 [00:03<00:00, 392.41frames/s]

100%|██████████| 2412/2412 [00:24<00:00, 100.26frames/s]

100%|██████████| 2412/2412 [00:06<00:00, 391.45frames/s]

100%|██████████| 1308/1308 [00:15<00:00, 82.62frames/s]

100%|██████████| 1308/1308 [00:08<00:00, 161.91frames/s]

100%|██████████| 1320/1320 [00:14<00:00, 91.09frames/s]

100%|██████████| 1320/1320 [00:04<00:00, 265.21frames/s]

100%|██████████| 1200/1200 [00:16<00:00, 70.75frames/s]

100%|██████████| 1200/1200 [00:25<00:00, 46.45frames/s]

100%|██████████| 1068/1068 [00:09<00:00, 108.38frames/s]

100%|██████████| 1068/1068 [00:03<00:00, 292.68frames/s]

100%|██████████| 720/720 [00:08<00:00, 85.93frames/s]

100%|██████████| 720/7

In [ ]:
data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

,reference,transcription,translation
0,шаҳри романтикӣ ва назаррабои синтра дар шимол ва дар наздикии дастрас ҷойгир аст ки баъди нақли олиҷаноби аз ҷониби лорд байрон сабтшуда барои хориҷиён машҳур гардидааст,"Шехри романтики и назарабои Синтра дар шимол и дар наздеки Дасрас джойгир аст, ки баади нақле олижаноби аз жониби Лорд Байрон цабдшуда, барају хариджиян машхур гардида аз.","The city of Sintra is located to the north and near Dasrath, which is later known as Lord Byron."
1,вақте ки дониши забони юнонӣ коҳиш ёфт ғарб аз решаҳои фалсафӣ ва илмии юнонӣ бурида шуд,"Бақт е ки даниши забони ёнони каўиш ёфт, гарб аз решахоји фалсафије ва ѫлми ёнони борида шуд.","When he learned the Greek language, the West was taken over by the Greek philosophies and sciences."
2,бемориро хукҳо мегузаронанд ки он баъд тавассути хомӯшакҳо ба одамон мегузарад,"Бемариро хукха мегузаронан, ки ан бад тавасоти хамушакха ба одамон мегузарад.","Illness is passed on by judges, who then pass it on to people through silences."
3,хонум кирхнер нияти худро ҷиҳати пешниҳоди номзадии худ ба мансаби президенти театри аргентина эълон кард ки ӯ маъракаи пешазинтихоботии худро соли 2005 барои сенат ҳамчун узви ҳайати музофоти буэнос-айрес дар ҳамон ҷо оғоз карда буд,"Ханом Кирхнер нияти худра джиати пеш ниходи намзади худ ба мансаби презенти театри агенти елон карт, ки у маракаи пежатз интихаботи худра сали 2005 бараи Сенат хамчун озви хайати музофоти Буенос Айрес дар хамончо агоз карда буд.",Ms. Kirkhner announced her intention to become a president of the Agenti Theater in 2005 for the Senate as well as as an additional member of the House of Representatives.
4,аҳолии ватикан тақрибан 800 нафарро ташкил медиҳад он хурдтарин кишвари мустақили ҷаҳон ва кишвари дорои шумораи камтарини аҳолӣ ба шумор меравад,Ахоли ватикан таqрибан 800 нафар оташкил меди хат. Он хуртарин кишвари мустаqили джехон ва кишвари дорои шмурек камтарин ахоли баш шмур ме рават.,"The population of the Vatican is approximately 800 people. It is the smallest independent country in the world, and has the lowest population in the country."
5,шаҳри мазкур инчунин пойгоҳе барои болобароӣ ба кӯҳи нирагонго ҳамроҳ бо баъзе арзонтарин пайгирии гориллаҳои кӯҳӣ дар африқо мебошад,Шахри Маскор инчунин паигохе барои балабарои ба кухи Нирагонга хамрох ба базе арзонтарин паигире гари лахой кухи Африка мебошад,The city of Mazkor is such a place for hiking to the Nira Ganga mountain with some of the cheapest hiking trails in Africa.
6,эҳтимолан кӯдакони одамгурез қабл аз партофта шудан ё гурехта рафтан ба муомилаи бераҳмона ё ҷароҳат дучор шудаанд,"Ехтимолан кудакони одамгурез, кабла аз партофта шудан ёо гурехтарафтан, бамо амилеи бирахмона ёо жарохат дучор шуда адд.","If people remain silent, children may have faced until the end of their life and been"
7,шоу-барномаи садоӣ ва чароғии пирамидаҳо яке аз ҷолибтарин чизҳои минтақаи мазкур барои кӯдакон аст,Шоу барномаи Садои и Чароги пирамидахо јаки аз джолептарин чесои минтақаи маскур бараји кудакон аст.,Today's program is one of the most beautiful things in the Mascour region for children.
8,касабланка яке аз ҷойҳои камҷолибтарин барои харид дар тамоми марокаш мебошад,Касабланка јак јас джојфоји кам джолиптарин бараји харид дар тамоми Марокаш ме бушат.,Casablanca is one of the most beautiful places to visit in Morocco.
9,яке аз маъмултарин усулҳои барои нишон додани аҳамияти иҷтимоӣ истифодашаванда ин ба кор бурдани якчанд ҳодисаҳои нохуш бо кӯдакон мебошад ки бинобар сабаби беэътиноӣ бадбахтӣ ё зӯроварии барқасдона аз ҷониби калонсолон дар синни балоғати худ бо ҷамъиятикунонӣ гирифта нашуда буданд,"Їаки аз маамолтарин усолхои барој нишандадан е ахамијат ештимаи истифодашаванда, ин ба карбурдани јакчан ходисахој нахуш ба кудакон мебушад, ки бинабар сабабе биетинаи бадбахти ја зурабари бархасдона аз жанеби калонсолон дар сини балаготи ход, ба жамијати конани грифта нашуда буданд.","One of the most common methods to show social importance is to use it to work 